In [10]:
from keras.models import load_model
from keras.applications.imagenet_utils import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [11]:
from keras.metrics import top_k_categorical_accuracy
def top_1_error(y_true, y_pred):
    return 1-top_k_categorical_accuracy(y_true, y_pred, k=1)

def top_3_error(y_true, y_pred):
    return 1-top_k_categorical_accuracy(y_true, y_pred, k=3)

In [12]:
def preprocess_img(img):
    import numpy as np
    img = np.expand_dims(img, axis=0)
    return preprocess_input(img)

datagen_test = ImageDataGenerator(preprocessing_function=preprocess_img)

def test_generator(batch_size, path):
    return datagen_test.flow_from_directory(path, target_size=(224, 224),
                                           batch_size=batch_size, 
                                           class_mode='categorical', 
                                           shuffle=False)

In [13]:
import pandas as pd
image_level_statistics = pd.read_csv('./results/data_analysis/image_level_statistics.csv')
n_test_images = (image_level_statistics['sample'] == 'test').sum()
print('We have {} test images'.format(n_test_images))

We have 4330 test images


In [14]:
batch_size = 10

# First, we investigate the benchmark performance on the test set.

## Last layer

In [6]:
bm_last = load_model('./saved_models/benchmark/weights_vgg16_top_only.hdf5', 
                     custom_objects={'top_1_error': top_1_error, 
                                     'top_3_error': top_3_error})

test_performance_bm_last = bm_last.evaluate_generator(
    test_generator(batch_size, path='./data/images/test'), 
    steps = n_test_images/batch_size)
print('Validation loss: {:2f}'.format(test_performance_bm_last[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*test_performance_bm_last[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*test_performance_bm_last[2]))

Found 4330 images belonging to 22 classes.
Validation loss: 2.010500
Validation top-1-error rate: 51.69%
Validation top-3-error rate: 24.94%


## Dense layers

In [7]:
bm_dense = load_model('./saved_models/benchmark/weights_vgg16_dense.hdf5', 
                     custom_objects={'top_1_error': top_1_error, 
                                     'top_3_error': top_3_error})

test_performance_bm_dense = bm_dense.evaluate_generator(
    test_generator(batch_size, path='./data/images/test'), 
    steps = n_test_images/batch_size)
print('Validation loss: {:2f}'.format(test_performance_bm_dense[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*test_performance_bm_dense[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*test_performance_bm_dense[2]))

Found 4330 images belonging to 22 classes.
Validation loss: 1.561630
Validation top-1-error rate: 46.37%
Validation top-3-error rate: 22.40%


## Last convolutional block and dense

In [8]:
bm_conv_and_dense = load_model('./saved_models/benchmark/weights_vgg16_conv_and_dense.hdf5', 
                     custom_objects={'top_1_error': top_1_error, 
                                     'top_3_error': top_3_error})

test_performance_bm_conv_and_dense = bm_conv_and_dense.evaluate_generator(
    test_generator(batch_size, path='./data/images/test'), 
    steps = n_test_images/batch_size)
print('Validation loss: {:2f}'.format(test_performance_bm_conv_and_dense[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*test_performance_bm_conv_and_dense[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*test_performance_bm_conv_and_dense[2]))

Found 4330 images belonging to 22 classes.
Validation loss: 1.502480
Validation top-1-error rate: 41.99%
Validation top-3-error rate: 18.48%


# Now we look at the two-stage model

## We have already cropped the images for the first stage in 5.
(That is all we need)

## Next, run the second stage

### Last layer

In [15]:
two_stage_last = load_model('./saved_models/f_rcnn/second_stage/weights_vgg16_top_only.hdf5', 
                     custom_objects={'top_1_error': top_1_error, 
                                     'top_3_error': top_3_error})

test_performance_two_stage_last = two_stage_last.evaluate_generator(
    test_generator(batch_size, path='./data/f_rcnn_second_stage/test'), 
    steps = n_test_images/batch_size)
print('Validation loss: {:2f}'.format(test_performance_two_stage_last[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*test_performance_two_stage_last[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*test_performance_two_stage_last[2]))

Found 4330 images belonging to 22 classes.
Validation loss: 1.913339
Validation top-1-error rate: 48.73%
Validation top-3-error rate: 24.06%


### Dense layers

In [16]:
two_stage_dense = load_model('./saved_models/f_rcnn/second_stage/weights_vgg16_dense.hdf5', 
                     custom_objects={'top_1_error': top_1_error, 
                                     'top_3_error': top_3_error})

test_performance_two_stage_dense = two_stage_dense.evaluate_generator(
    test_generator(batch_size, path='./data/f_rcnn_second_stage/test'), 
    steps = n_test_images/batch_size)
print('Validation loss: {:2f}'.format(test_performance_two_stage_dense[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*test_performance_two_stage_dense[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*test_performance_two_stage_dense[2]))

Found 4330 images belonging to 22 classes.
Validation loss: 1.504728
Validation top-1-error rate: 44.67%
Validation top-3-error rate: 20.62%


### Last convolutional block and dense

In [17]:
two_stage_conv_and_dense = load_model('./saved_models/f_rcnn/second_stage/weights_vgg16_conv_and_dense.hdf5', 
                     custom_objects={'top_1_error': top_1_error, 
                                     'top_3_error': top_3_error})

test_performance_two_stage_conv_and_dense = two_stage_conv_and_dense.evaluate_generator(
    test_generator(batch_size, path='./data/f_rcnn_second_stage/test'), 
    steps = n_test_images/batch_size)
print('Validation loss: {:2f}'.format(test_performance_two_stage_conv_and_dense[0]))
print('Validation top-1-error rate: {:.2f}%'.format(100*test_performance_two_stage_conv_and_dense[1]))
print('Validation top-3-error rate: {:.2f}%'.format(100*test_performance_two_stage_conv_and_dense[2]))

Found 4330 images belonging to 22 classes.
Validation loss: 1.370984
Validation top-1-error rate: 40.07%
Validation top-3-error rate: 17.25%
